# Import Required Libraries
Import necessary libraries such as NumPy, VTK, and vmtk.

In [23]:
# Import Required Libraries
import numpy as np
import vtk
import vmtk
from vmtk import vmtkscripts
from vmtk.vmtkscripts import vmtkCenterlines
from vmtk.vmtkscripts import vmtkPointListSeedSelector

import os

# Set the working directory
os.chdir(r"C:\Users\Rikke\OneDrive - Syddansk Universitet\6. semester\Bacehlor projekt\Broncho-Project\test")

# Verify the current working directory
print("Current working directory:", os.getcwd())

Current working directory: C:\Users\Rikke\OneDrive - Syddansk Universitet\6. semester\Bacehlor projekt\Broncho-Project\test


# Define Helper Functions
Define all the helper functions from the provided code, including `point_cloud_to_watertight_mesh`, `mesh_to_volume`, `visualize_mesh_with_centerline`, `slice_mesh_and_connect_centroids`, and others.

In [24]:
# Helper Functions

def point_cloud_to_watertight_mesh(point_cloud_file, output_mesh_file, alpha=0.1):
    """Converts a point cloud (.npy) to a watertight mesh using alpha shapes."""
    points = np.load(point_cloud_file)
    vtk_points = vtk.vtkPoints()
    for point in points:
        vtk_points.InsertNextPoint(point)
    poly_data = vtk.vtkPolyData()
    poly_data.SetPoints(vtk_points)
    delaunay = vtk.vtkDelaunay3D()
    delaunay.SetInputData(poly_data)
    delaunay.SetAlpha(alpha)
    delaunay.Update()
    surface_filter = vtk.vtkDataSetSurfaceFilter()
    surface_filter.SetInputConnection(delaunay.GetOutputPort())
    surface_filter.Update()
    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetFileName(output_mesh_file)
    writer.SetInputConnection(surface_filter.GetOutputPort())
    writer.Write()
    print(f"Watertight mesh saved to: {output_mesh_file}")

def mesh_to_volume(input_mesh_file, volume_file, spacing=(1.0, 1.0, 1.0)):
    """Converts a watertight mesh to a volumetric representation (e.g., voxel grid)."""
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(input_mesh_file)
    reader.Update()
    bounds = reader.GetOutput().GetBounds()
    image_data = vtk.vtkImageData()
    image_data.SetSpacing(spacing)
    dims = [
        int((bounds[1] - bounds[0]) / spacing[0]),
        int((bounds[3] - bounds[2]) / spacing[1]),
        int((bounds[5] - bounds[4]) / spacing[2]),
    ]
    image_data.SetDimensions(dims)
    image_data.SetOrigin(bounds[0], bounds[2], bounds[4])
    stencil = vtk.vtkPolyDataToImageStencil()
    stencil.SetInputConnection(reader.GetOutputPort())
    stencil.SetOutputSpacing(spacing)
    stencil.SetOutputOrigin(bounds[0], bounds[2], bounds[4])
    stencil.SetOutputWholeExtent(image_data.GetExtent())
    stencil.Update()
    volume = vtk.vtkImageStencil()
    volume.SetInputData(image_data)
    volume.SetStencilConnection(stencil.GetOutputPort())
    volume.ReverseStencilOff()
    volume.SetBackgroundValue(0)
    volume.Update()
    writer = vtk.vtkXMLImageDataWriter()
    writer.SetFileName(volume_file)
    writer.SetInputConnection(volume.GetOutputPort())
    writer.Write()
    print(f"Volume saved to: {volume_file}")

def clean_and_connect_mesh(input_mesh):
    """Cleans the mesh and extracts the largest connected component."""
    connectivity_filter = vtk.vtkConnectivityFilter()
    connectivity_filter.SetInputData(input_mesh)
    connectivity_filter.SetExtractionModeToLargestRegion()
    connectivity_filter.Update()
    return connectivity_filter.GetOutput()

def smooth_mesh(input_mesh, iterations=30, relaxation_factor=0.1):
    """Smooths the mesh to improve quality."""
    smoother = vtk.vtkSmoothPolyDataFilter()
    smoother.SetInputData(input_mesh)
    smoother.SetNumberOfIterations(iterations)
    smoother.SetRelaxationFactor(relaxation_factor)
    smoother.FeatureEdgeSmoothingOff()
    smoother.BoundarySmoothingOn()
    smoother.Update()
    return smoother.GetOutput()

def generate_voronoi_from_mesh(mesh_file, voronoi_output_file):
    """
    Generates a Voronoi diagram based on the points of a watertight mesh.
    """
    # Read the watertight mesh
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(mesh_file)
    reader.Update()
    mesh = reader.GetOutput()

    # Clean the mesh to ensure it is well-prepared
    cleaner = vtk.vtkCleanPolyData()
    cleaner.SetInputData(mesh)
    cleaner.Update()
    clean_mesh = cleaner.GetOutput()

    # Extract points from the cleaned mesh
    points = clean_mesh.GetPoints()

    # Create a polydata object to store the points
    point_polydata = vtk.vtkPolyData()
    point_polydata.SetPoints(points)

    # Generate the Delaunay tessellation (as a substitute for Voronoi)
    delaunay_filter = vtk.vtkDelaunay3D()
    delaunay_filter.SetInputData(point_polydata)
    delaunay_filter.Update()

    # Write the Delaunay tessellation to a file
    writer = vtk.vtkXMLUnstructuredGridWriter()
    writer.SetFileName(voronoi_output_file)
    writer.SetInputData(delaunay_filter.GetOutput())
    writer.Write()

    print(f"Voronoi diagram saved to: {voronoi_output_file}")

def extract_edges_from_voronoi(voronoi_file):
    """
    Extracts the edges from a Voronoi diagram.
    """
    # Read the Voronoi diagram
    reader = vtk.vtkXMLUnstructuredGridReader()
    reader.SetFileName(voronoi_file)
    reader.Update()
    voronoi = reader.GetOutput()

    # Extract the edges of the Voronoi diagram
    geometry_filter = vtk.vtkGeometryFilter()
    geometry_filter.SetInputData(voronoi)
    geometry_filter.Update()
    edges = geometry_filter.GetOutput()

    print(f"Number of points in edges: {edges.GetNumberOfPoints()}")
    print(f"Number of cells in edges: {edges.GetNumberOfCells()}")

    return edges

def compute_centroid_near_y(polydata, y_threshold, is_bottom=True, tolerance=2.0):
    """
    Compute the centroid of points near a given y-threshold.
    :param polydata: vtkPolyData object containing the mesh.
    :param y_threshold: The y-coordinate threshold (e.g., ymin or ymax).
    :param is_bottom: Whether to compute for the bottom (True) or top (False) region.
    :param tolerance: The tolerance for selecting points near the threshold.
    :return: Centroid as a list [x, y, z].
    """
    points = polydata.GetPoints()
    num_points = points.GetNumberOfPoints()
    selected_points = []

    for i in range(num_points):
        point = points.GetPoint(i)
        if is_bottom:
            # Select points strictly below y_threshold + tolerance
            if point[1] <= y_threshold + tolerance:
                selected_points.append(point)
        else:
            # Select points strictly above y_threshold - tolerance
            if point[1] >= y_threshold - tolerance:
                selected_points.append(point)

    print(f"Number of points selected near {'ymin' if is_bottom else 'ymax'}: {len(selected_points)}")

    if not selected_points:
        raise ValueError(f"No points found near the specified threshold (y_threshold={y_threshold}, tolerance={tolerance}).")

    # Compute the centroid of the selected points
    centroid = np.mean(selected_points, axis=0)
    return centroid


def extract_largest_region(polydata):
    """
    Extract the largest connected region from the mesh.
    :param polydata: vtkPolyData object containing the mesh.
    :return: vtkPolyData of the largest connected region.
    """
    connectivity_filter = vtk.vtkConnectivityFilter()
    connectivity_filter.SetInputData(polydata)
    connectivity_filter.SetExtractionModeToLargestRegion()
    connectivity_filter.Update()
    return connectivity_filter.GetOutput()

def visualize_points_on_mesh(mesh, points):
    """
    Visualize points on a mesh using VTK.
    :param mesh: vtkPolyData object containing the mesh.
    :param points: List of points to visualize.
    """
    vtk_points = vtk.vtkPoints()
    for point in points:
        vtk_points.InsertNextPoint(point)

    point_polydata = vtk.vtkPolyData()
    point_polydata.SetPoints(vtk_points)

    sphere_source = vtk.vtkSphereSource()
    sphere_source.SetRadius(0.01)
    glyph = vtk.vtkGlyph3D()
    glyph.SetInputData(point_polydata)
    glyph.SetSourceConnection(sphere_source.GetOutputPort())
    glyph.Update()

    mesh_mapper = vtk.vtkPolyDataMapper()
    mesh_mapper.SetInputData(mesh)
    mesh_actor = vtk.vtkActor()
    mesh_actor.SetMapper(mesh_mapper)

    glyph_mapper = vtk.vtkPolyDataMapper()
    glyph_mapper.SetInputConnection(glyph.GetOutputPort())
    glyph_actor = vtk.vtkActor()
    glyph_actor.SetMapper(glyph_mapper)
    glyph_actor.GetProperty().SetColor(1, 0, 0)  # Red points

    renderer = vtk.vtkRenderer()
    render_window = vtk.vtkRenderWindow()
    render_window.AddRenderer(renderer)
    render_window_interactor = vtk.vtkRenderWindowInteractor()
    render_window_interactor.SetRenderWindow(render_window)

    renderer.AddActor(mesh_actor)
    renderer.AddActor(glyph_actor)
    renderer.SetBackground(0.1, 0.1, 0.1)  # Dark background

    render_window.Render()
    render_window_interactor.Start()


def extract_centerlines_from_voronoi(voronoi_file, centerline_output_file, smooth_iterations=30, relaxation_factor=0.1):
    """
    Automatically extracts the centerlines from a Voronoi diagram.
    """
    # Extract edges from the Voronoi diagram
    edges = extract_edges_from_voronoi(voronoi_file)
    print(f"Number of points in edges: {edges.GetNumberOfPoints()}")
    print(f"Number of cells in edges: {edges.GetNumberOfCells()}")

    # Clean and triangulate the edges
    cleaner = vtk.vtkCleanPolyData()
    cleaner.SetInputData(edges)
    cleaner.Update()

    triangulator = vtk.vtkTriangleFilter()
    triangulator.SetInputConnection(cleaner.GetOutputPort())
    triangulator.Update()

    full_region = triangulator.GetOutput()
    bounds = full_region.GetBounds()
    print(f"Bounds of the geometry: {bounds}")

    # Compute source and target points based on y-coordinate
    try:
        source_point = compute_centroid_near_y(full_region, bounds[2], is_bottom=True, tolerance=2.0)  # ymin
        target_point = compute_centroid_near_y(full_region, bounds[3], is_bottom=False, tolerance=2.0)  # ymax

        # Ensure source and target points are distinct
        if np.allclose(source_point, target_point):
            raise ValueError("Source and target points are identical. Check the geometry or increase the tolerance.")

    except ValueError as e:
        print(f"Error computing centroids: {e}")
        print("Falling back to bounding box center points.")
        source_point = [(bounds[0] + bounds[1]) / 2, bounds[2], (bounds[4] + bounds[5]) / 2]  # Center at ymin
        target_point = [(bounds[0] + bounds[1]) / 2, bounds[3], (bounds[4] + bounds[5]) / 2]  # Center at ymax

    print(f"Source point: {source_point}")
    print(f"Target point: {target_point}")

    # Visualize the source and target points
    visualize_points_on_mesh(full_region, [source_point, target_point])

    # Convert source and target points to flat Python lists
    source_points_flat = list(source_point)
    target_points_flat = list(target_point)

    # Extract centerlines using vmtkCenterlines
    centerline_filter = vmtkCenterlines()
    centerline_filter.Surface = full_region
    centerline_filter.SourcePoints = source_points_flat
    centerline_filter.TargetPoints = target_points_flat
    centerline_filter.SeedSelectorName = 'pointlist'  # Explicitly set to pointlist to disable interaction
    centerline_filter.Execute()

    # Smooth the extracted centerlines
    print("Smoothing the centerlines...")
    smoother = vtk.vtkSmoothPolyDataFilter()
    smoother.SetInputData(centerline_filter.Centerlines)
    smoother.SetNumberOfIterations(smooth_iterations)
    smoother.SetRelaxationFactor(relaxation_factor)
    smoother.FeatureEdgeSmoothingOff()
    smoother.BoundarySmoothingOn()
    smoother.Update()
    smoothed_centerlines = smoother.GetOutput()

    # Save the smoothed centerlines
    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetFileName(centerline_output_file)
    writer.SetInputData(smoothed_centerlines)
    writer.Write()

    print(f"Smoothed centerlines saved to: {centerline_output_file}")

def extract_centerlines_from_surface_mesh(surface_mesh_file, centerline_output_file, smooth_iterations=30, relaxation_factor=0.1):
    """
    Extracts the centerlines directly from the surface mesh.
    """
    # Read the surface mesh
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(surface_mesh_file)
    reader.Update()
    surface_mesh = reader.GetOutput()

    # Compute bounds of the surface mesh
    bounds = surface_mesh.GetBounds()
    print(f"Bounds of the surface mesh: {bounds}")

    # Compute source and target points based on y-coordinate
    try:
        source_point = compute_centroid_near_y(surface_mesh, bounds[2], is_bottom=True, tolerance=2.0)  # ymin
        target_point = compute_centroid_near_y(surface_mesh, bounds[3], is_bottom=False, tolerance=2.0)  # ymax

        # Ensure source and target points are distinct
        if np.allclose(source_point, target_point):
            raise ValueError("Source and target points are identical. Check the geometry or increase the tolerance.")

    except ValueError as e:
        print(f"Error computing centroids: {e}")
        print("Falling back to bounding box center points.")
        source_point = [(bounds[0] + bounds[1]) / 2, bounds[2], (bounds[4] + bounds[5]) / 2]  # Center at ymin
        target_point = [(bounds[0] + bounds[1]) / 2, bounds[3], (bounds[4] + bounds[5]) / 2]  # Center at ymax

    print(f"Source point: {source_point}")
    print(f"Target point: {target_point}")

    # Visualize the source and target points
    visualize_points_on_mesh(surface_mesh, [source_point, target_point])

    # Convert source and target points to flat Python lists
    source_points_flat = list(source_point)
    target_points_flat = list(target_point)

    # Extract centerlines using vmtkCenterlines
    centerline_filter = vmtkCenterlines()
    centerline_filter.Surface = surface_mesh
    centerline_filter.SourcePoints = source_points_flat
    centerline_filter.TargetPoints = target_points_flat
    centerline_filter.SeedSelectorName = 'pointlist'  # Explicitly set to pointlist to disable interaction
    centerline_filter.Execute()

    # Smooth the extracted centerlines
    print("Smoothing the centerlines...")
    smoother = vtk.vtkSmoothPolyDataFilter()
    smoother.SetInputData(centerline_filter.Centerlines)
    smoother.SetNumberOfIterations(smooth_iterations)
    smoother.SetRelaxationFactor(relaxation_factor)
    smoother.FeatureEdgeSmoothingOff()
    smoother.BoundarySmoothingOn()
    smoother.Update()
    smoothed_centerlines = smoother.GetOutput()

    # Save the smoothed centerlines
    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetFileName(centerline_output_file)
    writer.SetInputData(smoothed_centerlines)
    writer.Write()

    print(f"Smoothed centerlines saved to: {centerline_output_file}")

def visualize_vtp(vtp_file):
    """
    Visualizes a .vtp file using VTK.
    """
    # Read the .vtp file
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(vtp_file)
    reader.Update()
    polydata = reader.GetOutput()

    # Create a mapper and actor
    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputData(polydata)
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)

    # Create a renderer, render window, and interactor
    renderer = vtk.vtkRenderer()
    render_window = vtk.vtkRenderWindow()
    render_window.AddRenderer(renderer)
    render_window_interactor = vtk.vtkRenderWindowInteractor()
    render_window_interactor.SetRenderWindow(render_window)

    # Add the actor to the renderer
    renderer.AddActor(actor)
    renderer.SetBackground(0.1, 0.1, 0.1)  # Dark background

    # Render and start interaction
    render_window.Render()
    render_window_interactor.Start()


def visualize_vtu(vtu_file):
    """
    Visualizes a .vtu file using VTK.
    """
    # Read the .vtu file
    reader = vtk.vtkXMLUnstructuredGridReader()
    reader.SetFileName(vtu_file)
    reader.Update()
    unstructured_grid = reader.GetOutput()

    # Create a mapper and actor
    mapper = vtk.vtkDataSetMapper()
    mapper.SetInputData(unstructured_grid)
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)

    # Create a renderer, render window, and interactor
    renderer = vtk.vtkRenderer()
    render_window = vtk.vtkRenderWindow()
    render_window.AddRenderer(renderer)
    render_window_interactor = vtk.vtkRenderWindowInteractor()
    render_window_interactor.SetRenderWindow(render_window)

    # Add the actor to the renderer
    renderer.AddActor(actor)
    renderer.SetBackground(0.1, 0.1, 0.1)  # Dark background

    # Render and start interaction
    render_window.Render()
    render_window_interactor.Start()

def visualize_mesh_with_centerline(mesh_file, centerline_file):
    """
    Visualizes the mesh and its centerline together using VTK.
    """
    # Read the mesh file
    mesh_reader = vtk.vtkXMLPolyDataReader()
    mesh_reader.SetFileName(mesh_file)
    mesh_reader.Update()
    mesh = mesh_reader.GetOutput()

    # Read the centerline file
    centerline_reader = vtk.vtkXMLPolyDataReader()
    centerline_reader.SetFileName(centerline_file)
    centerline_reader.Update()
    centerline = centerline_reader.GetOutput()

    # Create a mapper and actor for the mesh
    mesh_mapper = vtk.vtkPolyDataMapper()
    mesh_mapper.SetInputData(mesh)
    mesh_actor = vtk.vtkActor()
    mesh_actor.SetMapper(mesh_mapper)
    mesh_actor.GetProperty().SetColor(0.8, 0.8, 0.8)  # Light gray
    mesh_actor.GetProperty().SetOpacity(0.2)  # Semi-transparent

    # Create a mapper and actor for the centerline
    centerline_mapper = vtk.vtkPolyDataMapper()
    centerline_mapper.SetInputData(centerline)
    centerline_actor = vtk.vtkActor()
    centerline_actor.SetMapper(centerline_mapper)
    centerline_actor.GetProperty().SetColor(1.0, 0.0, 0.0)  # Red
    centerline_actor.GetProperty().SetLineWidth(3)

    # Create a renderer, render window, and interactor
    renderer = vtk.vtkRenderer()
    render_window = vtk.vtkRenderWindow()
    render_window.AddRenderer(renderer)
    render_window_interactor = vtk.vtkRenderWindowInteractor()
    render_window_interactor.SetRenderWindow(render_window)

    # Add the actors to the renderer
    renderer.AddActor(mesh_actor)
    renderer.AddActor(centerline_actor)
    renderer.SetBackground(0.1, 0.1, 0.1)  # Dark background

    # Render and start interaction
    render_window.Render()
    render_window_interactor.Start()

def slice_mesh_and_connect_centroids_with_branching(mesh_file, num_slices=30):
    """
    Slices the mesh into a specified number of slices along the Z-axis and connects centroids of each branch.
    Handles branching by identifying connected components in each slice.
    """
    # Read the mesh file
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(mesh_file)
    reader.Update()
    mesh = reader.GetOutput()

    # Get the bounds of the mesh
    bounds = mesh.GetBounds()
    print(f"Bounds of the mesh: {bounds}")
    z_min, z_max = bounds[4], bounds[5]
    print(f"Z-min: {z_min}, Z-max: {z_max}")
    slice_height = (z_max - z_min) / num_slices
    print(f"Slice height: {slice_height}")

    # Create a renderer, render window, and interactor
    renderer = vtk.vtkRenderer()
    render_window = vtk.vtkRenderWindow()
    render_window.AddRenderer(renderer)
    render_window_interactor = vtk.vtkRenderWindowInteractor()
    render_window_interactor.SetRenderWindow(render_window)

    # Add the original mesh to the renderer
    mesh_mapper = vtk.vtkPolyDataMapper()
    mesh_mapper.SetInputData(mesh)
    mesh_actor = vtk.vtkActor()
    mesh_actor.SetMapper(mesh_mapper)
    mesh_actor.GetProperty().SetColor(0.8, 0.8, 0.8)  # Light gray
    mesh_actor.GetProperty().SetOpacity(0.2)  # Semi-transparent
    renderer.AddActor(mesh_actor)

    # Initialize a dictionary to store centroids for each branch
    branch_centroids = {}

    # Slice the mesh and compute centroids for each branch
    for i in range(num_slices):
        z_lower = z_min + i * slice_height
        z_upper = z_lower + slice_height

        # Create a plane to slice the mesh
        plane = vtk.vtkPlane()
        plane.SetOrigin(0, 0, z_lower)
        plane.SetNormal(0, 0, 1)

        # Clip the mesh with the plane
        clipper = vtk.vtkClipPolyData()
        clipper.SetInputData(mesh)
        clipper.SetClipFunction(plane)
        clipper.Update()
        sliced_mesh = clipper.GetOutput()

        # Identify connected components in the slice
        connectivity_filter = vtk.vtkConnectivityFilter()
        connectivity_filter.SetInputData(sliced_mesh)
        connectivity_filter.SetExtractionModeToAllRegions()
        connectivity_filter.Update()
        labeled_mesh = connectivity_filter.GetOutput()

        # Get the number of regions (branches) in the slice
        num_regions = connectivity_filter.GetNumberOfExtractedRegions()
        print(f"Slice {i}: Found {num_regions} regions")

        for region_id in range(num_regions):
            # Extract the region
            connectivity_filter.SetExtractionModeToSpecifiedRegions()
            connectivity_filter.AddSpecifiedRegion(region_id)
            connectivity_filter.Update()
            branch_mesh = connectivity_filter.GetOutput()

            # Compute the centroid of the branch
            center_of_mass_filter = vtk.vtkCenterOfMass()
            center_of_mass_filter.SetInputData(branch_mesh)
            center_of_mass_filter.Update()
            centroid = center_of_mass_filter.GetCenter()

            # Store the centroid in the branch_centroids dictionary
            if region_id not in branch_centroids:
                branch_centroids[region_id] = []
            branch_centroids[region_id].append(centroid)
            print(f"Centroid of branch {region_id} at slice {i}: {centroid}")

        # Debug: Visualize the sliced mesh
        slice_mapper = vtk.vtkPolyDataMapper()
        slice_mapper.SetInputData(sliced_mesh)
        slice_actor = vtk.vtkActor()
        slice_actor.SetMapper(slice_mapper)
        #slice_actor.GetProperty().SetColor(0.0, 0.5, 0.0)  # Green
        slice_actor.GetProperty().SetOpacity(0.1)  # Semi-transparent
        renderer.AddActor(slice_actor)

    # Create polylines for each branch
    for branch_id, centroids in branch_centroids.items():
        points = vtk.vtkPoints()
        polyline = vtk.vtkPolyLine()
        polyline.GetPointIds().SetNumberOfIds(len(centroids))

        for i, centroid in enumerate(centroids):
            points.InsertNextPoint(centroid)
            polyline.GetPointIds().SetId(i, i)

        # Create a polydata to store the polyline
        polyline_data = vtk.vtkPolyData()
        polyline_data.SetPoints(points)
        cells = vtk.vtkCellArray()
        cells.InsertNextCell(polyline)
        polyline_data.SetLines(cells)

        # Add the polyline to the renderer
        polyline_mapper = vtk.vtkPolyDataMapper()
        polyline_mapper.SetInputData(polyline_data)
        polyline_actor = vtk.vtkActor()
        polyline_actor.SetMapper(polyline_mapper)
        polyline_actor.GetProperty().SetColor(1.0, 0.0, 0.0)  # Red
        polyline_actor.GetProperty().SetLineWidth(3)  # Increase line width
        renderer.AddActor(polyline_actor)

    # Set up the renderer
    renderer.SetBackground(0.1, 0.1, 0.1)  # Dark background

    # Render and start interaction
    render_window.Render()
    render_window_interactor.Start()

def point_cloud_to_vtp(point_cloud_file, output_vtp_file):
    """
    Converts a point cloud (.npy) to a .vtp file for visualization in VTK.
    """
    # Load the point cloud from the .npy file
    points = np.load(point_cloud_file)

    # Create a vtkPoints object and add the points to it
    vtk_points = vtk.vtkPoints()
    for point in points:
        vtk_points.InsertNextPoint(point)

    # Create a vtkPolyData object to store the points
    poly_data = vtk.vtkPolyData()
    poly_data.SetPoints(vtk_points)

    # Write the vtkPolyData to a .vtp file
    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetFileName(output_vtp_file)
    writer.SetInputData(poly_data)
    writer.Write()

    print(f"Point cloud saved to: {output_vtp_file}")

# Convert Point Cloud to Watertight Mesh
Run the `point_cloud_to_watertight_mesh` function with the specified input and output file paths.

In [25]:
# Input and output file paths
point_cloud_file = r"C:\Users\Rikke\OneDrive - Syddansk Universitet\6. semester\Bacehlor projekt\Broncho-Project\test\trimmed_pointcloud.npy"
output_mesh_file = r"C:\Users\Rikke\OneDrive - Syddansk Universitet\6. semester\Bacehlor projekt\Broncho-Project\test\output_mesh.vtp"

# Convert point cloud to watertight mesh
point_cloud_to_watertight_mesh(point_cloud_file, output_mesh_file, alpha=0.1)


Watertight mesh saved to: C:\Users\Rikke\OneDrive - Syddansk Universitet\6. semester\Bacehlor projekt\Broncho-Project\test\output_mesh.vtp


Visualize the watertight mesh

In [26]:
# Visualize the generated mesh
vtp_file = r"C:\Users\Rikke\OneDrive - Syddansk Universitet\6. semester\Bacehlor projekt\Broncho-Project\test\output_mesh.vtp"
# Check if the file exists and is not empty
if os.path.exists(vtp_file) and os.path.getsize(vtp_file) > 0:
    visualize_vtp(vtp_file)
else:
    print(f"Error: The file '{vtp_file}' is either empty or does not exist.")

# Generate Voronoi
Run the `generate_voronoi_from_mesh` function to extract the voronoi from the watertight mesh. 

In [27]:
# File paths
mesh_file = r"C:\Users\Rikke\OneDrive - Syddansk Universitet\6. semester\Bacehlor projekt\Broncho-Project\test\output_mesh.vtp"
voronoi_output_file = r"C:\Users\Rikke\OneDrive - Syddansk Universitet\6. semester\Bacehlor projekt\Broncho-Project\test\volumetric_mesh.vtu"

# Generate Voronoi diagram from watertight mesh
generate_voronoi_from_mesh(mesh_file, voronoi_output_file)

Voronoi diagram saved to: C:\Users\Rikke\OneDrive - Syddansk Universitet\6. semester\Bacehlor projekt\Broncho-Project\test\volumetric_mesh.vtu


Visualize the Voronoi diagram

In [28]:
# File path to the .vtu file
vtu_file = r"C:\Users\Rikke\OneDrive - Syddansk Universitet\6. semester\Bacehlor projekt\Broncho-Project\test\volumetric_mesh.vtu"

# Check if the file exists and is not empty
if os.path.exists(vtu_file) and os.path.getsize(vtu_file) > 0:
    visualize_vtu(vtu_file)
else:
    print(f"Error: The file '{vtu_file}' is either empty or does not exist.")

# Extract Centerline
Run the `extract_centerline_from_voronoi` function to extract the centerline from the voronoi.

In [29]:
voronoi_file = r"C:\Users\Rikke\OneDrive - Syddansk Universitet\6. semester\Bacehlor projekt\Broncho-Project\test\volumetric_mesh.vtu"
centerline_output_file = r"C:\Users\Rikke\OneDrive - Syddansk Universitet\6. semester\Bacehlor projekt\Broncho-Project\test\centerline.vtp"

extract_centerlines_from_voronoi(voronoi_file, centerline_output_file)

Number of points in edges: 4167
Number of cells in edges: 8330
Number of points in edges: 4167
Number of cells in edges: 8330
Bounds of the geometry: (-0.21030956506729126, 0.003435080638155341, -0.26695504784584045, 0.16247622668743134, 5.1693339347839355, 5.369777202606201)
Number of points selected near ymin: 4167
Number of points selected near ymax: 4167
Error computing centroids: Source and target points are identical. Check the geometry or increase the tolerance.
Falling back to bounding box center points.
Source point: [-0.10343724221456796, -0.26695504784584045, 5.269555568695068]
Target point: [-0.10343724221456796, 0.16247622668743134, 5.269555568695068]
Cleaning surface.
Triangulating surface.
Computing centerlines.
Computing centerlines...Smoothing the centerlines...
Smoothed centerlines saved to: C:\Users\Rikke\OneDrive - Syddansk Universitet\6. semester\Bacehlor projekt\Broncho-Project\test\centerline.vtp


In [30]:
# File paths
surface_mesh_file = r"C:\Users\Rikke\OneDrive - Syddansk Universitet\6. semester\Bacehlor projekt\Broncho-Project\test\output_mesh.vtp"
centerline_output_file = r"C:\Users\Rikke\OneDrive - Syddansk Universitet\6. semester\Bacehlor projekt\Broncho-Project\test\centerline_s.vtp"

# Extract centerlines directly from the surface mesh
extract_centerlines_from_surface_mesh(surface_mesh_file, centerline_output_file)

Bounds of the surface mesh: (-0.21030956506729126, 0.003435080638155341, -0.26695504784584045, 0.16247622668743134, 5.1693339347839355, 5.369777202606201)
Number of points selected near ymin: 7352
Number of points selected near ymax: 7352
Error computing centroids: Source and target points are identical. Check the geometry or increase the tolerance.
Falling back to bounding box center points.
Source point: [-0.10343724221456796, -0.26695504784584045, 5.269555568695068]
Target point: [-0.10343724221456796, 0.16247622668743134, 5.269555568695068]
Cleaning surface.
Triangulating surface.
Computing centerlines.
Computing centerlines...Smoothing the centerlines...
Smoothed centerlines saved to: C:\Users\Rikke\OneDrive - Syddansk Universitet\6. semester\Bacehlor projekt\Broncho-Project\test\centerline_s.vtp


# Visualize Mesh and Centerline
Run the `visualize_mesh_with_centerline` function to visualize the mesh and its centerline together.

In [31]:
# Visualize the mesh and centerline together
centerline_output_file = r"C:\Users\Rikke\OneDrive - Syddansk Universitet\6. semester\Bacehlor projekt\Broncho-Project\test\centerline.vtp"
visualize_mesh_with_centerline(output_mesh_file, centerline_output_file)

# Visualize the mesh and centerline together
centerline_output_file = r"C:\Users\Rikke\OneDrive - Syddansk Universitet\6. semester\Bacehlor projekt\Broncho-Project\test\centerline_s.vtp"
visualize_mesh_with_centerline(output_mesh_file, centerline_output_file)

# Slice Mesh and Connect Centroids
Run the `slice_mesh_and_connect_centroids` function to slice the mesh and connect centroids with a line.

In [32]:
# Slice the mesh and connect centroids
slice_mesh_and_connect_centroids_with_branching(output_mesh_file, num_slices=10)

Bounds of the mesh: (-0.21030956506729126, 0.003435080638155341, -0.26695504784584045, 0.16247622668743134, 5.1693339347839355, 5.369777202606201)
Z-min: 5.1693339347839355, Z-max: 5.369777202606201
Slice height: 0.020044326782226562
Slice 0: Found 4 regions
Centroid of branch 0 at slice 0: (-0.09472463212071469, -0.020006132879339816, 5.261223915741415)
Centroid of branch 1 at slice 0: (-0.09472463212071469, -0.020006132879339816, 5.261223915741415)
Centroid of branch 2 at slice 0: (-0.09472463212071469, -0.020006132879339816, 5.261223915741415)
Centroid of branch 3 at slice 0: (-0.09472463212071469, -0.020006132879339816, 5.261223915741415)
Slice 1: Found 3 regions
Centroid of branch 0 at slice 1: (-0.09364854347123248, -0.01961039110272899, 5.260614107454535)
Centroid of branch 1 at slice 1: (-0.09364854347123248, -0.01961039110272899, 5.260614107454535)
Centroid of branch 2 at slice 1: (-0.09364854347123248, -0.01961039110272899, 5.260614107454535)
Slice 2: Found 3 regions
Centroid

# Convert Point Cloud to VTP
Run the `point_cloud_to_vtp` function to convert the point cloud to a `.vtp` file.

In [33]:
# Convert the point cloud to .vtp
point_cloud_to_vtp(point_cloud_file, output_mesh_file)



Point cloud saved to: C:\Users\Rikke\OneDrive - Syddansk Universitet\6. semester\Bacehlor projekt\Broncho-Project\test\output_mesh.vtp
